In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import torch.optim as optim

In [2]:
import pandas as pd
import numpy as np

In [20]:
from dataloader import  missing_data, missing_data_rbf, eval_model, eval_bi_model
from models.model import MGRU,BiMGRU, train_MGRU, train_BiMGRU

In [4]:
df = pd.read_csv("./dataset/"+"pm25_missing.txt").drop(["datetime"], axis = 1)
dataset = missing_data_rbf(df, 64)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
model = MGRU(36, 64)
model.to(device)

MGRU(
  (temp_decay_h): TemporalDecay(
    (relu): ReLU()
  )
  (temp_decay_x): TemporalDecay(
    (relu): ReLU()
  )
  (output_layer): Linear(in_features=64, out_features=36, bias=True)
  (z_layer): FeatureRegression()
  (beta_layer): Linear(in_features=72, out_features=36, bias=True)
  (grucell): GRUCell(72, 64)
)

In [11]:
8756 // 64

136

In [12]:
for data in dataset:
    print(data.size())

torch.Size([64, 3, 64, 36])
torch.Size([64, 3, 64, 36])
torch.Size([8, 3, 64, 36])


In [15]:
loss_list = train_MGRU(model, 0.001,100, dataset, device)

loss: 104.32968139648438: 100%|██████████| 100/100 [01:12<00:00,  1.38it/s]


In [16]:
Nonscale_imputataion = eval_model(model, "pm25_ground.csv", "pm25_missing.csv")

Scale MAE : tensor(0.1908)
Scale MRE : tensor(0.2754)
Original MAE : 15.360264944542333
Original MRE : 0.21576326222445763


In [17]:
model2 = BiMGRU(36, 64)
model2.to(device)

BiMGRU(
  (fmGRU): MGRU(
    (temp_decay_h): TemporalDecay(
      (relu): ReLU()
    )
    (temp_decay_x): TemporalDecay(
      (relu): ReLU()
    )
    (output_layer): Linear(in_features=64, out_features=36, bias=True)
    (z_layer): FeatureRegression()
    (beta_layer): Linear(in_features=72, out_features=36, bias=True)
    (grucell): GRUCell(72, 64)
  )
  (bmGRU): MGRU(
    (temp_decay_h): TemporalDecay(
      (relu): ReLU()
    )
    (temp_decay_x): TemporalDecay(
      (relu): ReLU()
    )
    (output_layer): Linear(in_features=64, out_features=36, bias=True)
    (z_layer): FeatureRegression()
    (beta_layer): Linear(in_features=72, out_features=36, bias=True)
    (grucell): GRUCell(72, 64)
  )
)

In [18]:
loss_list = train_BiMGRU(model2, 0.001,100, dataset, device)

loss: 220.26470947265625, f_MGRU loss : 109.3384017944336, b_MGRU loss : 0.08653923124074936, consistency Loss : 110.83976745605469: 100%|██████████| 100/100 [02:16<00:00,  1.36s/it]


In [21]:
Nonscale_imputataion = eval_bi_model(model2, "pm25_ground.csv", "pm25_missing.csv")

Scale MAE : tensor(0.5123)
Scale MRE : tensor(0.7396)
Original MAE : 41.26353870669775
Original MRE : 0.5796225360973133


In [9]:
Nonscale_imputataion = eval_model(model, "pm25_ground.csv", "pm25_missing.csv")

Scale MAE : tensor(0.2237)
Scale MRE : tensor(0.3229)
Original MAE : 17.981988019045428
Original MRE : 0.2525902001220954


In [10]:
Nonscale_imputataion

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,137.999998,89.000000,104.999999,98.000000,109.000001,87.000000,88.000000,91.000000,87.000000,87.000000,...,69.262223,84.000000,116.999999,67.363484,96.999999,87.000000,74.000000,94.000000,107.070701,115.017861
1,124.000001,85.000000,121.000002,107.000000,101.000000,99.000000,104.999999,101.999999,103.000000,94.000000,...,84.608519,76.999999,108.999999,83.692972,96.999999,84.000001,84.000000,101.000000,97.214794,110.385838
2,127.000002,88.000000,129.999998,115.000000,102.000000,109.000000,114.000000,108.000000,112.000001,109.000000,...,95.991138,90.000000,105.000000,94.601606,103.000000,82.999999,100.000000,112.000000,106.046900,121.870493
3,129.000000,99.999999,136.999999,122.999999,108.000000,117.999999,118.000000,108.999999,117.000001,111.000000,...,99.828433,94.000001,105.000000,95.826133,106.999999,88.000000,103.000001,120.000000,107.410592,125.106039
4,111.912592,109.000000,143.999999,129.000000,115.000000,124.000001,129.999999,116.000001,123.999998,114.000000,...,101.366482,80.000000,103.999999,97.725838,110.999999,85.000000,108.000000,125.000000,109.553915,128.602624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,86.000000,72.000000,70.000000,72.000000,73.000000,63.000000,65.000000,69.000000,74.000000,65.999999,...,94.894990,90.824987,79.000000,69.000000,74.000000,125.000003,94.000000,67.000001,64.000001,66.729725
8755,87.000000,73.000000,73.000001,75.999999,83.000001,63.000000,61.000000,62.000001,62.000001,68.000001,...,92.566394,87.155220,64.000000,67.000000,75.000000,124.000003,128.000000,103.000000,69.000000,66.568798
8756,80.000000,74.000000,80.000000,70.000000,84.000000,69.000000,72.000000,75.000000,77.000000,73.000000,...,94.001082,95.405078,72.000000,70.000000,82.000000,125.000003,139.000000,166.000001,77.999999,71.062419
8757,90.294533,74.725859,87.000000,73.313409,76.624333,76.000000,79.000000,82.689084,82.000000,76.621252,...,92.629487,90.050429,89.644663,74.000000,86.013496,83.363710,83.193227,100.033523,88.824713,80.787288
